In [10]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# # Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

fatal: destination path 'detectron2' already exists and is not an empty directory.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment


In [11]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

zsh:1: command not found: nvcc
torch:  2.2 ; cuda:  2.2.2
detectron2: 0.6


In [15]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, random, cv2
# from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

matplotlib data path: /Users/muriellemardenli/opt/anaconda3/envs/axon-detection_venv/lib/python3.12/site-packages/matplotlib/mpl-data
CONFIGDIR=/Users/muriellemardenli/.matplotlib
interactive is False
platform is darwin
CACHEDIR=/Users/muriellemardenli/.matplotlib
Using fontManager instance from /Users/muriellemardenli/.matplotlib/fontlist-v390.json


In [29]:
# Register Dataset
# Must be run everytime you train ONLY ONCE
from detectron2.data.datasets import register_coco_instances
register_coco_instances("../src/data-coco/annotations/json_annotation_train.json", {}, "../src/data-coco/annotations/json_annotation_train.json", "../src/data-coco/images/train")
register_coco_instances("../src/data-coco/annotations/json_annotation_val.json", {},"../src/data-coco/annotations/json_annotation_val.json" , "../src/data-coco/images/val")
print('\n METADATA CATALOG')
print(list(detectron2.data.MetadataCatalog))


 METADATA CATALOG
['coco_2014_train', 'coco_2014_val', 'coco_2014_minival', 'coco_2014_valminusminival', 'coco_2017_train', 'coco_2017_val', 'coco_2017_test', 'coco_2017_test-dev', 'coco_2017_val_100', 'keypoints_coco_2014_train', 'keypoints_coco_2014_val', 'keypoints_coco_2014_minival', 'keypoints_coco_2014_valminusminival', 'keypoints_coco_2017_train', 'keypoints_coco_2017_val', 'keypoints_coco_2017_val_100', 'coco_2017_train_panoptic_separated', 'coco_2017_train_panoptic_stuffonly', 'coco_2017_train_panoptic', 'coco_2017_val_panoptic_separated', 'coco_2017_val_panoptic_stuffonly', 'coco_2017_val_panoptic', 'coco_2017_val_100_panoptic_separated', 'coco_2017_val_100_panoptic_stuffonly', 'coco_2017_val_100_panoptic', 'lvis_v1_train', 'lvis_v1_val', 'lvis_v1_test_dev', 'lvis_v1_test_challenge', 'lvis_v0.5_train', 'lvis_v0.5_val', 'lvis_v0.5_val_rand_100', 'lvis_v0.5_test', 'lvis_v0.5_train_cocofied', 'lvis_v0.5_val_cocofied', 'cityscapes_fine_instance_seg_train', 'cityscapes_fine_sem_s

In [30]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("../src/data-coco/annotations/json_annotation_train.json",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 60    # number of epochs
# 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.DEVICE = "cpu"  # Force CPU usage
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config /Users/muriellemardenli/opt/anaconda3/envs/axon-detection_venv/lib/python3.12/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[09/30 10:41:37 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[09/30 10:41:37 d2.engine.train_loop]: Starting training from iteration 0


/Users/muriellemardenli/opt/anaconda3/envs/axon-detection_venv/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403213615/work/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[09/30 10:58:01 d2.utils.events]:  eta: 0:27:04  iter: 19  total_loss: 3.416  loss_cls: 1.97  loss_box_reg: 1.232    time: 47.3028  last_time: 48.3471  data_time: 0.4865  last_data_time: 0.0165   lr: 7.9337e-05  
[09/30 11:16:53 d2.utils.events]:  eta: 0:14:00  iter: 39  total_loss: 1.038  loss_cls: 0.3575  loss_box_reg: 0.6732    time: 46.9037  last_time: 39.5909  data_time: 0.0070  last_data_time: 0.0022   lr: 0.00016259  
Saving checkpoint to ./output/model_final.pth
[09/30 11:34:34 d2.utils.events]:  eta: 0:00:00  iter: 59  total_loss: 0.8959  loss_cls: 0.2918  loss_box_reg: 0.6161    time: 46.6257  last_time: 38.9885  data_time: 0.0075  last_data_time: 0.0013   lr: 0.00024584  
[09/30 11:34:34 d2.engine.hooks]: Overall training speed: 58 iterations in 0:45:04 (46.6257 s / it)
[09/30 11:34:34 d2.engine.hooks]: Total training time: 0:45:05 (0:00:01 on hooks)


In [16]:
%load_ext tensorboard
%tensorboard --logdir output

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 45269), started 10:35:19 ago. (Use '!kill 45269' to kill it.)

In [34]:
import os

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[09/30 11:49:16 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
[Checkpointer] Loading from ./output/model_final.pth ...


In [35]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("../src/data-coco/annotations/json_annotation_val.json", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "../src/data-coco/annotations/json_annotation_val.json")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[09/30 11:49:18 d2.data.datasets.coco]: Loaded 10 images in COCO format from ../src/data-coco/annotations/json_annotation_val.json
[09/30 11:49:18 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[09/30 11:49:18 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/30 11:49:18 d2.data.common]: Serializing 10 elements to byte tensors and concatenating them all ...
[09/30 11:49:18 d2.data.common]: Serialized dataset takes 0.39 MiB
[09/30 11:49:18 d2.evaluation.evaluator]: Start inference on 10 batches
[09/30 11:49:31 d2.evaluation.evaluator]: Inference done 1/10. Dataloading: 5.8781 s/iter. Inference: 7.2257 s/iter. Eval: 0.0005 s/iter. Total: 13.1056 s/iter. ETA=0:01:57
[09/30 11:49:39 d2.evaluation.evaluator]: Inference done 2/10. Dataloading: 2.9395 s/iter. Inference: 7.5566 s/iter. Eval: 0.0005 s/iter. Total: 10.4978 s/i

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/muriellemardenli/opt/anaconda3/envs/axon-detection_venv/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/muriellemardenli/opt/anaconda3/envs/axon-detection_venv/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/Users/muriellemardenli/Desktop/mainn/A2024/NeuroPoly/NeuroPolyProjects/axon-detection/notebooks/detectron2/detectron2/data/common.py", line 125, in __getitem__
    data = self._map_func(self._dataset[cur_idx])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/muriellemardenli/Desktop/mainn/A2024/NeuroPoly/NeuroPolyProjects/axon-detection/notebooks/detectron2/detectron2/data/datas